In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag, map_tag
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import TextBlob
from afinn import Afinn

In [2]:
df=pd.read_csv('health_news.csv')

In [3]:
df.head()

,Unnamed: 0,numbers,time,tweets
0,0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...


In [4]:
df.drop(['numbers', 'time'], axis=1, inplace=True)

In [5]:
df.shape

(55230, 2)

In [6]:
df.isnull().values.any()

False

In [7]:
# import re
# import string
from Scripts.preprocessor import TwitterPreprocessor

In [8]:
df['clean_text'] = df.tweets.apply(lambda x: TwitterPreprocessor(x).ml_preprocess().text)

In [9]:
df.head()

,Unnamed: 0,tweets,clean_text
0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised
1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll
2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater
3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising
4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors


In [10]:
import boto3

In [27]:
df1=  df[:5000]
df1.shape

(5000, 3)

In [20]:
def create_sentiment_aws(row):
    """Uses AWS Comprehend to Create Sentiments on a DataFrame"""

    try:
      comprehend = boto3.client(service_name='comprehend', region_name="us-east-2")
      payload = comprehend.detect_sentiment(Text=row, LanguageCode='en')  
      sentiment = payload['Sentiment']
    except Exception:
      print("Size exceeded:  Fail")
      return None
    return sentiment

In [28]:
def apply_sentiment_aws(df, column="clean_text"):
    """Uses Pandas Apply to Create Sentiment Analysis"""

    df['Sentiment'] = df[column].apply(create_sentiment_aws)
    return df

In [29]:
df1_aws = apply_sentiment_aws(df1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
df1_aws.head()

,Unnamed: 0,tweets,clean_text,Sentiment
0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [32]:
df1_aws.to_csv('sentiment_for_healthtweets.csv', index='id')

In [29]:
sent_df = pd.read_csv('sentiment_for_healthtweets.csv')

In [30]:
sent_df.head()

,Unnamed: 0,Unnamed: 0.1,tweets,clean_text,Sentiment
0,0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [31]:
sent_df.index = sent_df.index.rename('id')
sent_df.head()

,Unnamed: 0,Unnamed: 0.1,tweets,clean_text,Sentiment
id,,,,,
0,0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised,NEUTRAL
1,1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll,NEUTRAL
2,2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater,MIXED
3,3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising,POSITIVE
4,4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors,NEUTRAL


In [32]:
sent_df =sent_df[["clean_text", "Sentiment"]]
sent_df.head()

,clean_text,Sentiment
id,,
0,breast cancer risk test devised,NEUTRAL
1,gp workload harming care bma poll,NEUTRAL
2,short people heart risk greater,MIXED
3,new approach hiv promising,POSITIVE
4,coalition undermined nhs doctors,NEUTRAL


In [33]:
sent_df.shape

(5000, 2)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X = sent_df.clean_text
y = sent_df.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [36]:
print(X_train.shape)
print(X_test.shape)

(3500,)
(1500,)


In [37]:
y_test.value_counts()

NEUTRAL     1069
NEGATIVE     258
POSITIVE     103
MIXED         70
Name: Sentiment, dtype: int64

# using Naive bayes classifier

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [40]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [41]:
%%time
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7166666666666667
              precision    recall  f1-score   support

       MIXED       0.00      0.00      0.00        70
    NEGATIVE       1.00      0.02      0.04       258
     NEUTRAL       0.72      1.00      0.83      1069
    POSITIVE       1.00      0.01      0.02       103

    accuracy                           0.72      1500
   macro avg       0.68      0.26      0.22      1500
weighted avg       0.75      0.72      0.60      1500

Wall time: 132 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Linear Svc

In [42]:
from sklearn.calibration import CalibratedClassifierCV

In [43]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='log', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                      

In [44]:
%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7133333333333334
              precision    recall  f1-score   support

       MIXED       0.00      0.00      0.00        70
    NEGATIVE       1.00      0.00      0.01       258
     NEUTRAL       0.71      1.00      0.83      1069
    POSITIVE       0.00      0.00      0.00       103

    accuracy                           0.71      1500
   macro avg       0.43      0.25      0.21      1500
weighted avg       0.68      0.71      0.59      1500

Wall time: 92.1 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [47]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.7506666666666667
              precision    recall  f1-score   support

       MIXED       0.39      0.23      0.29        70
    NEGATIVE       0.55      0.43      0.48       258
     NEUTRAL       0.81      0.89      0.85      1069
    POSITIVE       0.58      0.51      0.55       103

    accuracy                           0.75      1500
   macro avg       0.58      0.51      0.54      1500
weighted avg       0.73      0.75      0.74      1500

Wall time: 123 ms
